# Bluesky & databroker demonstration

 - Shows how to use `bluesky` to generate documents
 - Shows how to get the documents back from the `DataBroker`

## Event model

https://nsls-ii.github.io/architecture-overview.html
https://nsls-ii.github.io/bluesky/documents.html

Durring operation the `RunEngine` emits four kinds of documents

 - `Start` : what we know at the start of a 'run' (who, what, where, why)
 - `Event` : 'per-point' data
 - `Descriptor` : schema for the event documents
 - `Stop` : exit state of a 'run'

There may be many `Event` per `Descriptor` and many `Descritor` per `Start`. Callbacks can be used to subscribe to any or all of these documents while the experiment is running.

### Setup and configuration

In practice, this set up is setup in startup scripts, but shown here is full detail so that the notebook is stand-alone.

In [ ]:
%matplotlib widget
# general imports
import os
# import ipympl
import matplotlib.pyplot as plt
import numpy as np

# fitting library
import lmfit

# bluesky imports
import bluesky.plans as bp
import bluesky.callbacks as bc
import bluesky.utils as bu
from bluesky import RunEngine
# get a synthetic motor
from bluesky.examples import motor
# and the class for a synthentic detector
from bluesky.examples import SynGauss

# databorker imports
# Use a local sqlite + mongoquery based MDS to ease 
from portable_mds.sqlite.mds import MDS
from databroker import Broker

# to re-normalize the document
from doct import ref_doc_to_uid

# set up databroker 
dirname = os.path.expanduser('data-cache/')
mds = MDS({'directory': dirname,
            'timezone': 'America/Chicago'})
db = Broker(mds, None)



# set up matplotlib integration

plt.ion()
bu.install_nb_kicker()

# create a synthetic detector
noisy_det = SynGauss('noisy_det', motor, 'motor', center=0, Imax=100,
                     noise='poisson', sigma=1)

det = SynGauss('det', motor, 'motor', center=1, Imax=1, sigma=1)
# tweak det to have non-zero exposure time
det.exposure_time = .1


In [ ]:
# Create a run engine
RE = RunEngine({'purpose': 'demo', 'location': 'Chicago'})
# 
RE.subscribe('all', db.mds.insert)

## Important things in the namespace

The `RunEngine`

In [ ]:
RE

The `DataBroker` which is subscribed to `RE` to record measurements.

In [ ]:
db

Some synthetic devices.  The reading on both `det` and `noisy_det` are a function of the position of `motor`.  At beamlines, these would be `ophyd` objects.

In [ ]:
motor, det, noisy_det

Module of plans and callbacks

In [ ]:
bp, bc

## Basics

The two positional arguements to `RE.__call__` are the plan and callbacks, all extra keyword arguements are bundled into the start document.

In [ ]:
# print out the document names when they come out
RE(bp.count([det], num=7), 
   lambda name, doc: print(name),
   sample='synthetic'
)

### Documents back from the DataBroker

Get the most recent 'header'

In [ ]:
h = db[-1]

Look at the `Start`, `Stop`, and `Descriptor` documents

In [ ]:
h.start

In [ ]:
h.stop

In [ ]:
h.descriptors[0]

Get the `Event` documents

In [ ]:
# `get_events` is a generator, listify it to get concrete copies of everything
evs = list(db.get_events(h))

In [ ]:
# do not de-reference the descriptor to make more readable in html repr
ref_doc_to_uid(evs[1], 'descriptor')

### Some more interesting callbacks

In [ ]:
# A live table that updates during the experiment
RE(bp.scan([det], motor, -3, 3, 15), [bc.LiveTable(['det', 'motor'])])

In [ ]:
RE(bp.scan([det], motor, -1, 3, 25), [bc.LiveTable(['det', 'motor']), bc.LivePlot('det', 'motor')])

#### Get a table of data

In [ ]:
tab = db.get_table(db[-1])
tab

In [ ]:
tab.describe()

### Adaptive fitting plan

In [ ]:
def errorbar(lmfit_result, param_name):
    # width of 95% conf interfal:
    ci = lmfit_result.conf_interval()
    return ci[param_name][-2][1] - ci[param_name][1][1]


def gaussian(x, A, sigma, x0):
    return A * np.exp(-(x - x0)**2 / (2 * sigma**2))


model = lmfit.Model(gaussian)
guess = {'A': 10,
         'x0': 1,
         'sigma': lmfit.Parameter('sigma', 3, min=0)}


def scan_gaussian(detectors, motor, start, stop, num, *, ax=None,
                  err_thresh=0.07):

    if ax is None:
        ax = plt.gca()
    main_detector = detectors[0]
    main_motor_field, *_ = motor.describe()
    lf = bc.LiveFit(model, main_detector.name, {'x': main_motor_field}, guess)
    lfp = bc.LiveFitPlot(lf, color='r', ax=ax)
    lp = bc.LivePlot(main_detector, main_motor_field,
                     linestyle='none', marker='o', ax=ax)
    jitter = np.abs(stop - start) / (num * 10)
    @bp.subs_decorator([lfp, lp])
    @bp.stage_decorator(list(detectors) + [motor])
    @bp.run_decorator()
    def plan():
        while True:
            for step in np.linspace(start, stop, num):
                step = step + (jitter * np.random.randn(1)[0])
                yield from bp.abs_set(motor, step, wait=True)
                yield from bp.trigger_and_read(list(detectors) + [motor])
                yield from bp.checkpoint()
          
            if lf.result is None or errorbar(lf.result, 'sigma') < err_thresh:
                break

    return (yield from plan())

In [ ]:
plt.figure();

In [ ]:
RE(scan_gaussian([noisy_det], motor, -4, 4, 25, ax=plt.gca()))

Look at a pretty-printed table of the measurements

In [ ]:
db.process(db[-1], bc.LiveTable(['motor', 'noisy_det']))

Plot the measurements against sequence number

In [ ]:
plt.figure()
db.process(db[-1], bc.LivePlot('noisy_det', ax=plt.gca()))

Plot against motor position

In [ ]:
plt.figure()
db.process(db[-1], bc.LivePlot('noisy_det', 'motor', ax=plt.gca(), ls='none', marker='o'))

In [ ]:
db.get_table(db[-1])

## Searching the Data broker

https://nsls-ii.github.io/databroker/searching.html

In [ ]:
### by uid / partial uid

In [ ]:
h = db[-1]
assert db[h.start['uid']] == h
assert db[h.start['uid'][:5]] == h

Insert a bunch of runs with a varying metadata

In [ ]:
det.exposure_time = 0

def mega_plan(operator):
    base_md = {'operator': operator}
    for sample in ('A', 'B', 'C'):
        yield from bp.count([det], md={**base_md, 'role': 'calibration', 'sample': sample})
        yield from bp.scan([det], motor, -1, 3, 10, md={**base_md, 'role': 'ascan', 'sample': sample})

In [ ]:
RE(mega_plan('tcaswell'))

In [ ]:
RE(mega_plan('scampbell'))

In [ ]:
RE(mega_plan('dallan'))

### Search by keyword

In [ ]:
len(db(operator='dallan'))

In [ ]:
len(db(operator='dallan', role='calibration'))

In [ ]:
len(db(operator='dallan', role='calibration', sample='A'))

In [ ]:
len(db())